# Training of a CNN model for counting diamond shapes with different sizes in binary images

This notebook trains a CNN model for the Counting simple shapes (circles, squares or diamonds) experiment, more specifically diamond shapes with different sizes.

### Settings

In [ ]:
# Imports
from CNNcount import shape_images as si
import numpy as np
import matplotlib.pyplot as plt
from numpy import load
import os.path

import keras
from keras.utils import np_utils

In [ ]:
# filename for loading the data from the NPZ files (NumPy compressed)
same_shape_diff_radii_fname = "/home/elena/eStep/XAI/Data/CountingShapes/diamonds_diff_radii_60k.npz"

diff_shapes_same_radius_fname = "/home/elena/eStep/XAI/Data/CountingShapes/diff_shapes_same_radius_60k.npz"
diff_shapes_diff_radii_fname = "/home/elena/eStep/XAI/Data/CountingShapes/diff_shapes_diff_radii_60k.npz"

### Loading of pre-generated data and formatting of the data and labels.

In [ ]:
# input image dimensions and number of classes
img_rows, img_cols = 64, 64
num_classes = 3

#### Data formatting

In [ ]:
# load the set of NIM images with the same type and same radius and split to train, test and validaiton subsets
if os.path.isfile(same_shape_same_radius_fname): # already generated- just load
    print ("The file containing images of the same shape (circle) with same radius already exist!")
    # load from NPZ file for display
    images_train, images_val, images_test, labels_train, labels_val, labels_test = \
                                                                si.load_split_data(same_shape_same_radius_fname)
    
    
    if keras.backend.image_data_format() == 'channels_first':
        images_train = images_train.reshape(images_train.shape[0], 1, img_rows, img_cols)
        images_val = images_val.reshape(images_val.shape[0], 1, img_rows, img_cols)
        images_test = images_test.reshape(images_test.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        input_shape = (img_rows, img_cols, 1)
    print("Size of training data: ", np.shape(images_train), "and labels: ", np.shape(labels_train))
    print("Size of validation data: ", np.shape(images_val), "and labels: ", np.shape(labels_val))
    print("Size of testing data: ", np.shape(images_test), "and labels: ", np.shape(labels_test))
else: # missing data
    print ("The file containing images of the same shape (circle) with same radius does not exist!")
    print("Use the GenerateShapeImages notebook to generate the experimental data.") 

In [ ]:
# plot random 12 of the train images
si.plot_12images(images_train, labels_train)

#### Labels formatting.

In [ ]:
# convert class vectors to binary class matrices
labels_train = np_utils.to_categorical(labels_train-1, num_classes=None)
labels_val = np_utils.to_categorical(labels_val-1, num_classes=None)
labels_test = np_utils.to_categorical(labels_test-1, num_classes=None)

In [ ]:
print(labels_train)
print('labels_train shape:', labels_train.shape)
print('labels_val shape:', labels_val.shape)
print('labels_test shape:', labels_test.shape)

### CNN model.

In [ ]:
# imports
from CNNcount import model_count_shapes as mcs

In [ ]:
# parameters of the training
batch_size = 200
epochs = 3

In [ ]:
# generate the model
model = mcs.generate_cnncount_model(input_shape, num_classes)

print(model.summary())

In [ ]:
# train 
mcs.train_cnncount_model(model, images_train, labels_train,images_val, labels_val, batch_size, epochs)


#### Evaluate on test images

In [ ]:
score = model.evaluate(images_test, labels_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

#### Save the model

In [ ]:
# filename for model saving
same_shape_same_radius_model_fname = "/home/elena/eStep/XAI/Data/CountingShapes/model_circles_same_radius.h5"

In [ ]:
# save the trained model
model.save(same_shape_same_radius_model_fname)
print("Saved model to disk")
